# Plan-and-Solve 智能体实战

本笔记本演示了另外一种经典的智能体范式：**Plan-and-Solve (先规划，后执行)**。

### 核心思想：
与 ReAct 的“走一步看一步”不同，Plan-and-Solve 采用“三思而后行”的策略：
1. **规划阶段 (Planner)**：将复杂问题一次性拆解为多个逻辑连贯的子任务。
2. **执行阶段 (Executor)**：严格按照计划顺序，逐一解决子任务，并进行状态管理。

### C/C++ 类比：
这就像是 **编译器的工作流程**：
- **Planner**：类比为 **Frontend/Parser**，将源代码（问题）解析并生成中间表示（Intermediate Representation，即计划列表）。
- **Executor**：类比为 **Backend/Generator**，根据中间表示生成机器码（最终结果），每一步生成都依赖于符号表（历史状态）的更新。

In [1]:
import os
import ast
from llm_client import HelloAgentsLLM
from dotenv import load_dotenv

# 1. 环境与库导入
# 类比 C++ 的 #include 与环境变量设置
load_dotenv(dotenv_path="../../.env")

# 初始化 LLM 客户端
llm_client = HelloAgentsLLM()

In [2]:
# --- 2. 规划器 (Planner) 定义 ---
# 负责将大目标拆解为原子任务

PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划，```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

class Planner:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def plan(self, question: str) -> list[str]:
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        messages = [{"role": "user", "content": prompt}]
        
        print("--- 正在生成计划 ---")
        response_text = self.llm_client.think(messages=messages) or ""
        
        # 解析 LLM 输出的 Python 列表模式字符串
        # 类比 C++ 的字符串解析或脚本解释逻辑
        try:
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            # ast.literal_eval 类比 C++ 的安全 json/script 解析，避免直接执行代码
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        except Exception as e:
            print(f"❌ 解析计划时出错: {e}")
            return []

In [3]:
# --- 3. 执行器 (Executor) 定义 ---
# 负责线性执行计划，并维护状态历史

EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""

class Executor:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def execute(self, question: str, plan: list[str]) -> str:
        history = ""
        final_answer = ""
        
        print("\n--- 正在执行计划 ---")
        for i, step in enumerate(plan, 1):
            print(f"\n-> 正在执行步骤 {i}/{len(plan)}: {step}")
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question, plan=plan, history=history if history else "无", current_step=step
            )
            messages = [{"role": "user", "content": prompt}]
            
            response_text = self.llm_client.think(messages=messages) or ""
            
            # 维护历史记录（State Management），类比 C++ 的上下文状态机更新
            # 将每一步的结果存入 history，作为下一步的输入
            history += f"步骤 {i}: {step}\n结果: {response_text}\n\n"
            final_answer = response_text
            
        return final_answer

In [4]:
# --- 4. 智能体 (Agent) 整合 ---

class PlanAndSolveAgent:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client
        self.planner = Planner(self.llm_client)
        self.executor = Executor(self.llm_client)

    def run(self, question: str):
        # 第一阶段：规划 (Compiler Frontend)
        plan = self.planner.plan(question)
        if not plan:
            print("无法生成有效的行动计划。")
            return
            
        # 第二阶段：执行 (Compiler Backend)
        final_answer = self.executor.execute(question, plan)
        print(f"\n--- 任务完成 ---\n最终答案: {final_answer}")

# --- 运行测试 ---
agent = PlanAndSolveAgent(llm_client)
# 这是一个经典的需要多步逻辑推理的数学题
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
agent.run(question)

--- 正在生成计划 ---
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
```python
["步骤1: 确定周一卖出的苹果数量为15个。",
 "步骤2: 计算周二卖出的苹果数量，等于周一的两倍，即15 * 2。",
 "步骤3: 计算周三卖出的苹果数量，比周二少5个，即周二的数量 - 5。",
 "步骤4: 将周一、周二和周三卖出的苹果数量相加，得到总数。",
 "步骤5: 输出三天总共卖出的苹果数量。"]
```

--- 正在执行计划 ---

-> 正在执行步骤 1/5: 步骤1: 确定周一卖出的苹果数量为15个。
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
15

-> 正在执行步骤 2/5: 步骤2: 计算周二卖出的苹果数量，等于周一的两倍，即15 * 2。
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
30

-> 正在执行步骤 3/5: 步骤3: 计算周三卖出的苹果数量，比周二少5个，即周二的数量 - 5。
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
25

-> 正在执行步骤 4/5: 步骤4: 将周一、周二和周三卖出的苹果数量相加，得到总数。
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
70

-> 正在执行步骤 5/5: 步骤5: 输出三天总共卖出的苹果数量。
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
70

--- 任务完成 ---
最终答案: 70
